# Importar datos


In [5]:
import pandas as pd

df = pd.read_csv('data/data_2015_clean.csv')


In [15]:
df

,InfoDate,EmployeeID,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,...,Gender,JobLevel,JobRole,MaritalStatus,retirementType,resignationReason,Attrition,retirementDate,JobInvolvement,PerformanceRating
0,2015-12-31,2532,32,9,43270,8.0,13,0,4,0,...,Male,4,Sales Executive,Single,NaN,NaN,No,NaN,3,3
1,2015-12-31,1062,32,9,43270,8.0,13,0,4,0,...,Male,4,Sales Executive,Single,NaN,NaN,No,NaN,3,3
2,2015-12-31,4002,32,9,43270,8.0,13,0,4,0,...,Male,4,Sales Executive,Single,NaN,NaN,No,NaN,3,3
3,2015-12-31,421,48,1,21770,4.0,18,0,4,0,...,Male,1,Research Scientist,Married,NaN,NaN,No,NaN,3,3
4,2015-12-31,1891,48,1,21770,4.0,18,0,4,0,...,Male,1,Research Scientist,Married,NaN,NaN,No,NaN,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,2015-12-31,2745,50,2,83760,5.0,14,3,2,27,...,Female,4,Research Scientist,Divorced,NaN,NaN,No,NaN,3,3
4349,2015-12-31,4215,50,2,83760,5.0,14,3,2,27,...,Female,4,Research Scientist,Divorced,NaN,NaN,No,NaN,3,3
4350,2015-12-31,168,50,10,47870,0.0,12,0,3,27,...,Female,3,Manager,Married,NaN,NaN,No,NaN,3,3
4351,2015-12-31,3108,50,10,47870,0.0,12,0,3,27,...,Female,3,Manager,Married,NaN,NaN,No,NaN,3,3


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4353 entries, 0 to 4352
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   InfoDate                 4353 non-null   object 
 1   EmployeeID               4353 non-null   int64  
 2   Age                      4353 non-null   int64  
 3   DistanceFromHome         4353 non-null   int64  
 4   MonthlyIncome            4353 non-null   int64  
 5   NumCompaniesWorked       4353 non-null   float64
 6   PercentSalaryHike        4353 non-null   int64  
 7   StockOptionLevel         4353 non-null   int64  
 8   TrainingTimesLastYear    4353 non-null   int64  
 9   YearsAtCompany           4353 non-null   int64  
 10  EnvironmentSatisfaction  4353 non-null   int64  
 11  JobSatisfaction          4353 non-null   int64  
 12  WorkLifeBalance          4353 non-null   int64  
 13  BusinessTravel           4353 non-null   object 
 14  Department              

# Obtener dummies

In [41]:
df2 = pd.get_dummies(df.drop(['Attrition','retirementDate','retirementType','resignationReason','InfoDate'], axis=1))
df2['target'] = df['Attrition']

In [49]:
pd.set_option('display.max_columns',None)

In [50]:
df2

,EmployeeID,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsAtCompany,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobLevel,JobInvolvement,PerformanceRating,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_Bachelor,Education_Below College,Education_College,Education_Doctor,Education_Master,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,target
0,2532,32,9,43270,8.0,13,0,4,0,4,4,0,4,3,3,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,0
1,1062,32,9,43270,8.0,13,0,4,0,4,4,0,4,3,3,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,0
2,4002,32,9,43270,8.0,13,0,4,0,4,4,0,4,3,3,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,0
3,421,48,1,21770,4.0,18,0,4,0,4,3,0,1,3,3,False,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,0
4,1891,48,1,21770,4.0,18,0,4,0,4,3,0,1,3,3,False,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,2745,50,2,83760,5.0,14,3,2,27,2,3,3,4,3,3,False,False,True,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,0
4349,4215,50,2,83760,5.0,14,3,2,27,2,3,3,4,3,3,False,False,True,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,0
4350,168,50,10,47870,0.0,12,0,3,27,2,3,0,3,3,3,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,0
4351,3108,50,10,47870,0.0,12,0,3,27,2,3,0,3,3,3,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,0


In [42]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4353 entries, 0 to 4352
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EmployeeID                         4353 non-null   int64  
 1   Age                                4353 non-null   int64  
 2   DistanceFromHome                   4353 non-null   int64  
 3   MonthlyIncome                      4353 non-null   int64  
 4   NumCompaniesWorked                 4353 non-null   float64
 5   PercentSalaryHike                  4353 non-null   int64  
 6   StockOptionLevel                   4353 non-null   int64  
 7   TrainingTimesLastYear              4353 non-null   int64  
 8   YearsAtCompany                     4353 non-null   int64  
 9   EnvironmentSatisfaction            4353 non-null   int64  
 10  JobSatisfaction                    4353 non-null   int64  
 11  WorkLifeBalance                    4353 non-null   int64

# Binarizar variable objetivo

In [43]:
df2.target.replace({'Yes':1, 'No':0}, inplace=True)

# Separar variable objetivo de los datos

In [44]:
X = df2.drop('target',axis=1)
y = df2['target']

print(X.shape)
print(y.shape)

(4353, 46)
(4353,)


In [45]:
from sklearn.feature_selection import SelectKBest, f_classif,chi2
def select_kbest(X,y,score_f,k):
    sel_kb = SelectKBest(score_func=score_f,k=k)
    sel_kb.fit(X,y)
    new_cols= sel_kb.get_support()
    print('Score:\n', sel_kb.scores_, '\n P-values:\n', sel_kb.pvalues_)
    return new_cols

In [46]:
X_new= select_kbest(X,y,f_classif,30)
# Nuevo conjunto de datos
df_anova = X.iloc[:,X_new]
df_anova.head()

TypeError: '<' not supported between instances of 'str' and 'int'